In [1]:
import flair

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:

flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
s = flair.data.Sentence(txt)
flair_sentiment.predict(s)
total_sentiment = s.labels
total_sentiment
s.tag, s.score
flair_sentiment.predict(s);
s.labels
# from flair.data_fetcher import NLPTask
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

sec_corpus = flair.datasets.NER_ENGLISH_SEC_FILLINGS()
print(sec_corpus.obtain_statistics())
word_embeddings = [WordEmbeddings('glove'),
                   FlairEmbeddings('news-forward'),
                   FlairEmbeddings('news-backward'),
                   ]
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings  #, Sentence

glove_embedding = WordEmbeddings('glove')
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

document_embeddings = DocumentPoolEmbeddings([glove_embedding,
                                              flair_embedding_backward,
                                              flair_embedding_backward],
                                             mode='min')
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                   hidden_size=512,
                                                                   reproject_words=True,
                                                                   reproject_words_dimension=256,
                                                                   )

classifier = TextClassifier(document_embeddings, label_type=, multi_label=False)

from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings

glove_embedding = WordEmbeddings('glove')

document_embeddings = DocumentRNNEmbeddings([glove_embedding])
sentence = Sentence('The grass is green . And the sky is blue .')

document_embeddings.embed(sentence)

print(sentence.get_embedding())
df

In [2]:
from flair.datasets import UD_ENGLISH
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [3]:
corpus = flair.datasets.NER_ENGLISH_SEC_FILLINGS()

2022-05-30 19:59:30,852 Reading data from /Users/zad/.flair/datasets/ner_english_sec_fillings
2022-05-30 19:59:30,853 Train: /Users/zad/.flair/datasets/ner_english_sec_fillings/FIN5.txt
2022-05-30 19:59:30,854 Dev: None
2022-05-30 19:59:30,855 Test: /Users/zad/.flair/datasets/ner_english_sec_fillings/FIN3.txt


In [6]:
label_type = 'pos'

In [7]:
label_dict = corpus.make_label_dictionary(label_type=label_type)

2022-05-30 20:10:39,500 Computing label dictionary. Progress:


1051it [00:00, 7905.41it/s]

2022-05-30 20:10:39,637 Dictionary created for label 'pos' with 42 values: NNP (seen 5463 times), NN (seen 4962 times), IN (seen 4239 times), DT (seen 3761 times), CC (seen 2055 times), NNS (seen 2039 times), , (seen 1866 times), JJ (seen 1706 times), CD (seen 1349 times), : (seen 1240 times), . (seen 1218 times), VB (seen 925 times), VBN (seen 915 times), TO (seen 885 times), VBZ (seen 678 times), RB (seen 613 times), MD (seen 440 times), VBG (seen 430 times), VBD (seen 330 times), VBP (seen 291 times)


In [9]:
# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

In [10]:
embeddings = StackedEmbeddings(embeddings=embedding_types)

In [12]:
# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

2022-05-30 20:11:14,066 SequenceTagger predicts: Dictionary with 42 tags: <unk>, NNP, NN, IN, DT, CC, NNS, ,, JJ, CD, :, ., VB, VBN, TO, VBZ, RB, MD, VBG, VBD, VBP, PRP$, POS, WDT, LS, PRP, NNPS, '', JJR, WRB, JJS, $, ``, -NONE-, PDT, EX, RBR, RP, WP$, -X-, WP, RBS


In [13]:
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

In [ ]:
# 7. start training
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2022-05-30 20:11:32,340 ----------------------------------------------------------------------------------------------------
2022-05-30 20:11:32,341 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=44, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2022-05-30 20:11:32,342 ----------------------------------------------------------------------------------------------------
2022-05-30 20:11:32,343 Corpus: "Corpus: 1051 train + 117 dev + 305 test sentences"
2022-05-30 20:11:32,344 ----------------------------------------------------------------------------------------------------
2022-05-30 20:11:32,345 P

In [ ]:
# load the model you trained
model = SequenceTagger.load('resources/taggers/example-upos/final-model.pt')

# create example sentence
sentence = Sentence('I love Berlin')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())